In [299]:
import pandas as pd 
import os 
import numpy as np 

In [300]:
centro = '8811456' #input("Introduza o centro: ")

Importação de ficheiros da pasta FTP

In [301]:


folder_path = 'C:\\Users\\1420844\\Downloads\\dados_centros'

files = [file for file in os.listdir(folder_path) if os.path.isfile(os.path.join(folder_path, file))]


combined_df = pd.DataFrame()


for file in files:
    file_path = os.path.join(folder_path, file)
    
    
    df = pd.read_csv(file_path, encoding="latin-1", delimiter=';')
    
    
    combined_df = pd.concat([combined_df, df], ignore_index=True)



selecção do centro e codigo postal 

In [302]:
# -- limpeza do dataset 
df = combined_df
df.columns=[x.lower().strip() for x in df.columns]
df.drop_duplicates()
df['centro'] = df['centro'].astype(str)
df['centro'] = df['centro'].str.strip()
df['giro'] = df['giro'].str.replace(' ', '').str.strip().str[:5]
df['cp'] = df['cp'].str.replace(' ', '').str.strip()
df['longitude'] = df['longitude'].str.replace('\n', '').str.strip()
df=df[df.latitude != '0']
df=df[df.latitude != '0.0']
df = df[df['latitude'] != '']
df=df[df.cod_t_even == 'I']
df = df[df['cp'].str.len() >= 8]

In [303]:
df=df[df['centro']==centro]


In [304]:
# validar os giros que devem ser usados no pit stop

giro_unique_ocorrencias = df.groupby('giro')['data_criacao'].nunique().reset_index(name='N_ocorrencias_giro')

# Contar o número de ocorrências únicas por 'loptica'
loptica_unique_ocorrencias = df.groupby('giro')['loptica'].nunique().reset_index(name='N_ocorrencias_loptica')

giros_pit_stop = pd.merge(giro_unique_ocorrencias, loptica_unique_ocorrencias, left_on='giro', right_on='giro', how='inner')

In [ ]:
# downloads_path = os.path.join(os.path.expanduser("~"), "Downloads")


# file_path = os.path.join(downloads_path, f'{centro}.xlsx')
# giros_pit_stop.to_excel(file_path, index=False)

# print(f"File saved at: {file_path}")

In [308]:
# -- considera apenas cp7 que ocorreram várias vezez--#

giro_unique_cp7 = df.groupby('cp')['data_criacao'].nunique().reset_index(name='N_Unique_cp7')

giro_unique_cp7 = giro_unique_cp7[giro_unique_cp7['N_Unique_cp7'] >= 1]

giro_unique_cp7=np.array(giro_unique_cp7['cp'].unique())

df = df[df['cp'].isin(giro_unique_cp7)]

In [309]:
#-- excluir cp7 efetuados por várias rotas --##

unique_routes_by_cp = df.groupby('cp')['giro'].nunique().reset_index(name='Unique_Routes_Count')

unique_routes_by_cp = unique_routes_by_cp.sort_values(by='Unique_Routes_Count', ascending=False)

unique_routes_by_cp_filtered = unique_routes_by_cp[unique_routes_by_cp['Unique_Routes_Count'] ==1]

lista_excl_cp7=np.array(unique_routes_by_cp_filtered['cp'].unique())

df = df[df['cp'].isin(lista_excl_cp7)]

cp7_unique_counts = df['cp'].value_counts()

In [310]:
# selecção de giros 
codigo_prestador='3100'
codigo_prestador_2=''
giros_excluir=['1000A','1000B']

giro_unique_counts = df.groupby('giro')['data_criacao'].nunique().reset_index(name='N_Unique_Data_Criacao')

giro_unique_counts = giro_unique_counts[
    ((giro_unique_counts['giro'].str[:4] == codigo_prestador) |
     (giro_unique_counts['giro'].str[:4] == codigo_prestador_2)) &
    (~giro_unique_counts['giro'].apply(lambda x: any(x.startswith(prefix) for prefix in giros_excluir)))
]



giro_unique_counts_average = giro_unique_counts['N_Unique_Data_Criacao'].mean()
giro_unique_counts_average

giro_unique_counts=giro_unique_counts[giro_unique_counts['N_Unique_Data_Criacao'] > 0]
giro_unique_counts

print(giro_unique_counts)

lista_val_giro=np.array(giro_unique_counts['giro'].unique())

print(lista_val_giro)

df = df[df['giro'].isin(lista_val_giro)]

     giro  N_Unique_Data_Criacao
72  3100B                      5
73  3100G                      5
74  3100H                      3
75  3100I                      1
76  3100J                      5
77  3100K                      5
78  3100M                      2
['3100B' '3100G' '3100H' '3100I' '3100J' '3100K' '3100M']


Dados Sig Postal

In [311]:
file_path = 'C:\\Users\\1420844\\Downloads\\sig'

files = [file for file in os.listdir(file_path) if os.path.isfile(os.path.join(file_path, file))]

for file in files:

    file_path = os.path.join(file_path, file)
    df_sig = pd.read_csv(file_path, encoding="latin-1", delimiter=';')
    


In [312]:
filtro_cp4 = np.array(giro_unique_counts['giro'].str[:4].unique())

In [313]:


df_sig.columns=[x.lower().strip() for x in df_sig.columns]

df_sig = df_sig.apply(lambda x: x.astype(str).str.replace(' ', '').str.strip() if pd.notnull(x.name) else x)

df_sig['cp7'] = df_sig['cp7'].astype(str)

df_sig['point_x'] = df_sig['point_x'].str.replace('\n', '').str.strip()
df_sig['point_y'] = df_sig['point_y'].str.replace('\n', '').str.strip()
df_sig['point_x'] = df_sig['point_x'].str.replace(',', '.').str.strip()
df_sig['point_y'] = df_sig['point_y'].str.replace(',', '.').str.strip()
df_sig=df_sig[df_sig.point_x != '0']
df_sig=df_sig[df_sig.point_y != '0']
df_sig=df_sig[df_sig.point_x != '0.0']
df_sig=df_sig[df_sig.point_y != '0.0']
df_sig = df_sig[df_sig['point_x'] != '']
df_sig = df_sig[df_sig['point_y'] != '']
df_sig = df_sig[df_sig['point_x'] != 'nan']
df_sig = df_sig[df_sig['point_y'] != 'nan']
df_sig = df_sig.dropna()


In [314]:
df_sig = df_sig[df_sig['cp4'].isin(filtro_cp4)]

In [315]:
import pandas as pd


df['data_criacao'] = pd.to_datetime(df['data_criacao'], dayfirst=True)

result = df.groupby(['cp', 'giro']).size().reset_index(name='count')

idx = result.groupby('cp')['count'].idxmax()

df_result = result.loc[idx].reset_index(drop=True)



In [316]:
merged_df = pd.merge(df_sig, df_result[['cp', 'giro']], how='left', left_on='cp7', right_on='cp')


merged_df.drop(['cp', 'cp3','cpalf'], axis=1, inplace=True)


In [317]:
selected_columns = ['cp7', 'cp4', 'point_x', 'point_y', 'giro']
selected_df = merged_df[selected_columns]


downloads_path = os.path.join(os.path.expanduser("~"), "Downloads")


file_path = os.path.join(downloads_path, f'{codigo_prestador}.xlsx')
selected_df.to_excel(file_path, index=False)

print(f"File saved at: {file_path}")

File saved at: C:\Users\1420844\Downloads\3100.xlsx
